In [1]:
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient
from azure.ai.ml.entities import Data, Datastore
from azure.ai.ml.entities import OneLakeDatastore, OneLakeArtifact
from azure.ai.ml.constants import AssetTypes, InputOutputModes
from azureml.fsspec import AzureMachineLearningFileSystem
import pandas as pd
import fsspec
import mltable

In [6]:
client = MLClient.from_config(credential=DefaultAzureCredential(), path=".")
ws = client.workspaces.get(name=client.workspace_name)
print(ws.name)

Found the config file in: /mnt/batch/tasks/shared/LS_root/mounts/clusters/e4s-4c32g/code/.azureml/config.json


ml_test


## Connect To OneLake


In [ ]:
store = OneLakeDatastore(
    name="KTDataLake",
    description="Datastore pointing to an Microsoft fabric artifact.",
    one_lake_workspace_name="6558a001-c860-4020-8ab1-1e5ae1f48456",
    endpoint="onelake.dfs.fabric.microsoft.com",
    artifact = OneLakeArtifact(
        name="1202190f-974b-4175-b882-5ca0c26760f4/Files",
        type="lake_house"
    )
)

client.create_or_update(store)

In [ ]:
# create paths to the data files
paths = [{"file": "wasbs://data@azuremlexampledata.blob.core.windows.net/titanic.csv"}]

# create an MLTable from the data files
tbl = mltable.from_delimited_files(
    paths=paths,
    delimiter=",",
    header=MLTableHeaders.all_files_same_headers,
    infer_column_types=True,
    include_path_column=False,
    encoding=MLTableFileEncoding.utf8,
)

In [9]:
data_set = client.data.get("nmsdata_onelakeV2", version="6")

tbl = mltable.load(data_set.path)

df = tbl.to_pandas_dataframe()
df

,시간,RU영문명,RRC 시도호,MAX_UE,AirMAC DL(MB),PRB DL(%),소비전력(Watt)
0,2024-04-13 06:10:00,NPKG30087S,1.0,0.0,0.00,0.80,181.92
1,2024-04-13 06:15:00,NPKG30087S,NaN,0.0,0.00,0.81,183.42
2,2024-04-13 06:20:00,NPKG30087S,2.0,0.0,0.01,0.80,182.25
3,2024-04-13 06:25:00,NPKG30087S,1.0,1.0,0.04,0.80,182.01
4,2024-04-13 06:30:00,NPKG30087S,NaN,0.0,0.00,0.80,182.11
...,...,...,...,...,...,...,...
1217628,2024-04-18 08:15:00,NRKG41054S,3.0,6.0,7.57,0.90,242.59
1217629,2024-04-18 08:20:00,NRKG41054S,6.0,5.0,38.55,1.17,243.75
1217630,2024-04-18 08:25:00,NRKG41054S,2.0,6.0,22.75,1.04,241.94
1217631,2024-04-18 08:30:00,NRKG41054S,4.0,8.0,61.44,1.20,243.38
